In [1]:
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt
import datetime
from pyCP_APR import CP_APR
from pyCP_APR.datasets import load_dataset

ImportError: cannot import name 'CP_APR' from 'pyCP_APR' (unknown location)

In [6]:
cp_apr = CP_APR(n_iters=10,
                random_state=42,
                verbose=1,
                method='torch',
                device='cpu',
                return_type='numpy')